# Análisis de Escritos según Ámbito 2: Parte II

## Rúbrica: Calidad de la argumentación  

## Definiciones previas (para efectos de esta pauta)  

- **Argumentos sólidos:** aquellos que fundamentan racionalmente la tesis. Incluyen datos, hechos, cifras, argumentos causales, deductivos, inductivos y opiniones debidamente justificadas.  
- **Argumentos débiles:** aquellos que muestran poca solidez lógica, tales como generalizaciones, argumentos de autoridad, falacias, entre otros.  
- **Argumentos afectivo-persuasivos:** apelaciones basadas en sentimientos, sensaciones, opiniones sin fundamento, o recursos emocionales dirigidos a influir en el lector (ej.: temores, deseos, supersticiones, falsas creencias).  

## Escala de evaluación  

### 3 puntos – Alto nivel de argumentación  
- Se presentan **al menos tres argumentos sólidos**, claramente vinculados con la tesis planteada.  
- Los argumentos son coherentes entre sí, bien estructurados y no se contradicen.  
- No se utilizan argumentos débiles ni apelaciones afectivo-persuasivas.  

### 2 puntos – Nivel medio de argumentación  
- Se presenta **un argumento sólido** acompañado de **dos o más argumentos débiles**, todos vinculados a la tesis.  
- Los argumentos débiles no invalidan el sólido, pero reducen la fuerza global de la argumentación.  
- No se incluyen argumentos afectivo-persuasivos.  

### 1 punto – Nivel básico de argumentación  
- Se presentan **tres o más argumentos débiles** relacionados con la tesis.  
- No hay presencia de argumentos sólidos que respalden de manera firme la postura.  
- No se incluyen argumentos afectivo-persuasivos.  

### 0 puntos – Ausencia o invalidez de la argumentación  
- El texto **no presenta argumentos** que respalden la tesis (puede consistir en una exposición descriptiva, una lista de oraciones sin relación lógica o enunciados de preguntas).  
- **O bien**, se incluye **al menos un argumento afectivo-persuasivo**, lo que invalida el criterio de solidez argumentativa en esta pauta.  


In [3]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

data_path = os.path.join("..", "data", "escritos_seleccionados_a1.xlsx")
df = pd.read_excel(data_path)
textos = df["escrito"].dropna().tolist()

prompt_calidad_arg = """
Quiero que evalúes la calidad de la argumentación de un texto según la siguiente rúbrica:

- Argumentos sólidos: fundamentan racionalmente la tesis (datos, hechos, cifras, razonamientos deductivos/inductivos/causales, opiniones justificadas).
- Argumentos débiles: poca solidez lógica (generalizaciones, argumentos de autoridad, falacias).
- Argumentos afectivo-persuasivos: apelaciones a emociones, sentimientos o creencias sin fundamento.

Tu respuesta debe ser un JSON válido, sin explicaciones adicionales, en el siguiente formato:

{{
  "errores": {{
        "argumentos_solidos": ["Primer argumento sólido", "Segundo argumento sólido", ...],
        "argumentos_debiles": ["Primer argumento débil", "Segundo argumento débil", ...],
        "argumentos_afectivos": ["Primer argumento afectivo", "Segundo argumento afectivo", ...],
        ...
  }}
}}

### Texto a evaluar:
{essay}

### INSTRUCCIONES CRÍTICAS:
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

resultados = []

for i, texto in enumerate(textos, start=1):    
    prompt_calidad = prompt_calidad_arg.format(essay=texto)

    print(f"Procesando texto {i}/{len(textos)}...")

    messages = [
        {"role": "system", "content": 'Realiza las tareas de corrección que te solicite para un texto entregado'},
        {"role": "user", "content": prompt_calidad}
    ]

    try:
        response = client.chat.completions.create(
            model=DEPLOYMENT_GPT,
            messages=messages,
            # temperature=0, (No es soportado en Azure OpenAI)
            response_format={"type": "json_object"}
        )

        respuesta_limpia = response.choices[0].message.content.strip()
        resultado_json = json.loads(respuesta_limpia)
        resultados.append({
            "id_texto": i,
            "resultado": resultado_json
        })

    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON en texto {i}: {e}")
        print("Respuesta cruda recibida:")
        print(respuesta_limpia)

output_dir = os.path.join("..", "resultados", "05")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "calidad_arg.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print(f"\nTodos los resultados guardados en {output_path}")

Procesando texto 1/20...
Procesando texto 2/20...
Procesando texto 3/20...
Procesando texto 4/20...
Procesando texto 5/20...
Procesando texto 6/20...
Procesando texto 7/20...
Procesando texto 8/20...
Procesando texto 9/20...
Procesando texto 10/20...
Procesando texto 11/20...
Procesando texto 12/20...
Procesando texto 13/20...
Procesando texto 14/20...
Procesando texto 15/20...
Procesando texto 16/20...
Procesando texto 17/20...
Procesando texto 18/20...
Procesando texto 19/20...
Procesando texto 20/20...

Todos los resultados guardados en ..\resultados\05\calidad_arg.json
